2024-09-19 - Ronen Shekel

In [ ]:
import matplotlib


matplotlib.use('TKAgg')
%load_ext autoreload
%autoreload 2
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots, get_imaging_masks
from pianoq_results.scan_result import ScanResult
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCSimResult
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.misc.mplt import mimshow, mplot
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_diffuser_masks
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
import time 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_19_filters_80nm'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])
U_no = 1
phases_no = 2 

# HW

In [ ]:
# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

# Motors
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)
print('Got Time tagger')

## correlations

In [ ]:
integration_time = 4
tt.set_integration_time(integration_time)
filter_type = '20nm'

# in the different Us different phases this seemed to work best 
masks_path = glob.glob(rf'{dir_path}\U{U_no}U*.masks')[0]
phases_path = glob.glob(rf'{dir_path}\U{phases_no}U*.phases')[0]

pr = PhaseFinderResult()
pr.loadfrom(phases_path)

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks

masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
masks = add_phase_input_spots(masks, pr.phases)
mplc.load_masks(masks, linear_tilts=True)

# locs 
locs_sig_path = glob.glob(fr'{dir_path}\U{U_no}U*sig.locs')[0]
locs_idl_path = glob.glob(fr'{dir_path}\U{U_no}U*idl.locs')[0]
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
scan_path = fr'{dir_path}\{timestamp}_filter_{filter_type}_exp_{integration_time}s.dscan'
mplc.restore_location()
dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, no_hw_mode=True)
dps.scan()

Plan: 
- Run correlations with 20 nm. 4s, then 20s. 
- Take from past the single scan 
- change filters to 80nm 
- run correlations with same all 
- reoptimize phases and try again 
- run singles scan